In [1]:
#import necessary libraries
import pandas as pd
import csv
import re
import validators
import emoji
import unidecode
import nltk
import pickle
nltk.download('stopwords')
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import WhitespaceTokenizer
from itertools import chain
import time
from sklearn.feature_extraction.text import TfidfVectorizer
import keras
from keras.preprocessing import sequence
import tensorflow as tf
from keras import Input
from keras import optimizers
from keras import backend as K
import torch
from keras import regularizers
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_blobs
from keras.models import Sequential
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib import pyplot
from keras.models import load_model
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectPercentile, f_classif

import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import np_utils
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from itertools import chain

from tqdm import tqdm
from gensim.models import fasttext
from gensim.test.utils import datapath
import os, re, csv, math, codecs, pickle, nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to C:\Users\Kirby
[nltk_data]     Wenceslao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#Load test and testresutls   

with open('x_test.pkl', 'rb') as file:
    x_test = pickle.load(file)
    
with open('y_test.pkl', 'rb') as file:
    y_test = pickle.load(file)
    
with open('fastTextCNN_test_results.pkl', 'rb') as file:
    fastTextCNN_test_results = pickle.load(file) 
    
with open('TFIDFFFNN_test_results.pkl', 'rb') as file:
    TFIDFFFNN_test_results = pickle.load(file)  
    
#Convert Labels from Strings to categorical Integers {Non-Hate = 1, Hate = 0}


df_ytest = pd.DataFrame(y_test, columns = ['Label'])

fastTextCNN_test_results.columns = ['Tokens', 'Label', 'Prediction']

fastTextCNN_tweets_joined = []

for i in range(len(fastTextCNN_test_results.Tokens.values)):
    tweet = " ".join(fastTextCNN_test_results.Tokens.values[i])
    fastTextCNN_tweets_joined.append(tweet)
    
fastTextCNN_test_results.insert(0, "Tweet", fastTextCNN_tweets_joined, True)


TFIDFFFNN_test_results.columns = ['Tokens', 'Label', 'Prediction']

tfidfffnn_tweets_joined = []
for i in range(len(TFIDFFFNN_test_results.Tokens.values)):
    tweet = " ".join(TFIDFFFNN_test_results.Tokens.values[i])
    tfidfffnn_tweets_joined.append(tweet)
    
TFIDFFFNN_test_results.insert(0, "Tweet", tfidfffnn_tweets_joined, True)
#TFIDFFFNN_test_results['Label'] = TFIDFFFNN_test_results['Label'].str.get(0)
#TFIDFFFNN_test_results['Prediction'] = TFIDFFFNN_test_results['Prediction'].str.get(0)
#flatten training set for fitting on tfidf
#x_train_fit = list(chain.from_iterable(x_train))

mapping = {'Positive': 0, 'Negative': 1, 'Neutral':2}
mapping_reverse = {0:'Positive', 1: 'Negative', 2: 'Neutral'}
fastTextCNN_test_results = fastTextCNN_test_results.replace({'Label': mapping_reverse})
TFIDFFFNN_test_results = TFIDFFFNN_test_results.replace({'Label': mapping_reverse})
df_ytest = df_ytest.replace({'Label': mapping})

In [3]:
#collect index of misclassified and remove from results


fasttext_misclassified = []
tfidf_misclassified = []

for i in range(len(y_test)):
    if(y_test[i] != fastTextCNN_test_results.Label.values[i]):
        fasttext_misclassified.append(i)
        
    if(y_test[i] != TFIDFFFNN_test_results.Label.values[i]):
        tfidf_misclassified.append(i)
    
print(len(fasttext_misclassified))
print(len(tfidf_misclassified))

#remove misclassified
fastTextCNN_test_results.drop(fasttext_misclassified, inplace = True )
TFIDFFFNN_test_results.drop(tfidf_misclassified, inplace = True )

print()

267
284



In [4]:
fastTextCNN_test_results.shape

(501, 4)

In [5]:
TFIDFFFNN_test_results.shape

(484, 4)

In [6]:
TFIDFFFNN_test_results

,Tweet,Tokens,Label,Prediction
0,bbmsara win rin inday sara win,"[bbmsara, win, rin, inday, sara, win]",Positive,0.891507
2,presidential aspirant manila fired shots fello...,"[presidential, aspirant, manila, fired, shots,...",Negative,0.763399
3,gobyernong tapat angat buhay si yancy iskolar ...,"[gobyernong, tapat, angat, buhay, si, yancy, i...",Positive,0.998858
4,oras kandidatong tunay magdadala pagbabago nau...,"[oras, kandidatong, tunay, magdadala, pagbabag...",Positive,0.969080
5,ganyan nalang kontra sasabihin bbm kase wala y...,"[ganyan, nalang, kontra, sasabihin, bbm, kase,...",Negative,0.999668
...,...,...,...,...
762,ready all-out ba eto starter pack ipanalo natin,"[ready, all-out, ba, eto, starter, pack, ipana...",Positive,0.996817
763,presidente bise presidente kiko pangilinan let...,"[presidente, bise, presidente, kiko, pangilina...",Positive,0.998056
764,mt mayon showed seems like looking vp like vp ...,"[mt, mayon, showed, seems, like, looking, vp, ...",Positive,0.564144
765,president,[president],Neutral,0.534855


In [7]:
TFIDFFFNN_test_results.shape

(484, 4)

In [8]:
print(len(df_ytest))

768


In [9]:
def dummy_fun(doc):
    return doc

def get_keywords(data,filename):
    print(filename)
    tfidf = TfidfVectorizer(
        analyzer='word',
        tokenizer=dummy_fun,
        preprocessor=dummy_fun,
        token_pattern=None)  

    #fit text to tfidf
    tfidf.fit_transform(data)


    fasttext_dict = list(zip(tfidf.get_feature_names(), tfidf.idf_))
    fasttext_df = pd.DataFrame(fasttext_dict, columns = ['Token', 'TFIDF_Value'])
    fasttextresults_label_sorted = fasttext_df.sort_values(by=['TFIDF_Value'], ascending=False)
    fasttextresults_label_sorted.to_csv(filename, index=False)
    
    return fasttextresults_label_sorted

In [10]:
TFIDFFFNN_test_results.to_csv('TFIDFFFNN_test_results.csv', index=False)
TFIDFFFNN_test_results

,Tweet,Tokens,Label,Prediction
0,bbmsara win rin inday sara win,"[bbmsara, win, rin, inday, sara, win]",Positive,0.891507
2,presidential aspirant manila fired shots fello...,"[presidential, aspirant, manila, fired, shots,...",Negative,0.763399
3,gobyernong tapat angat buhay si yancy iskolar ...,"[gobyernong, tapat, angat, buhay, si, yancy, i...",Positive,0.998858
4,oras kandidatong tunay magdadala pagbabago nau...,"[oras, kandidatong, tunay, magdadala, pagbabag...",Positive,0.969080
5,ganyan nalang kontra sasabihin bbm kase wala y...,"[ganyan, nalang, kontra, sasabihin, bbm, kase,...",Negative,0.999668
...,...,...,...,...
762,ready all-out ba eto starter pack ipanalo natin,"[ready, all-out, ba, eto, starter, pack, ipana...",Positive,0.996817
763,presidente bise presidente kiko pangilinan let...,"[presidente, bise, presidente, kiko, pangilina...",Positive,0.998056
764,mt mayon showed seems like looking vp like vp ...,"[mt, mayon, showed, seems, like, looking, vp, ...",Positive,0.564144
765,president,[president],Neutral,0.534855


In [11]:
groupedby_ffnn_label = TFIDFFFNN_test_results.groupby(TFIDFFFNN_test_results.Label)

#group by label
ffnn_positive = groupedby_ffnn_label.get_group("Positive")
ffnn_negative = groupedby_ffnn_label.get_group("Negative")
ffnn_neutral = groupedby_ffnn_label.get_group("Neutral")

In [12]:
#tfidf highest keywords ffnn positive
get_keywords(ffnn_positive.Tokens.values,'TFIDFFFNN_positive_keywords.csv').head(20)

TFIDFFFNN_positive_keywords.csv


,Token,TFIDF_Value
0,accomplishments,5.543295
572,pamilya,5.543295
545,oscar,5.543295
546,overflowing,5.543295
547,owner,5.543295
548,p.s,5.543295
549,pack,5.543295
550,paet,5.543295
551,pag,5.543295
555,pagboboto,5.543295


In [13]:
#tfidf highest keywords ffnn negative
get_keywords(ffnn_negative.Tokens.values,'TFIDFFFNN_negative_keywords.csv').head(20)

TFIDFFFNN_negative_keywords.csv


,Token,TFIDF_Value
0,aattend,5.448516
862,napakalaking,5.448516
860,napakadali,5.448516
859,nanood,5.448516
858,naninira,5.448516
857,nangyari,5.448516
856,nangangampanya,5.448516
855,nang,5.448516
854,nanay,5.448516
851,namumuhay,5.448516


In [14]:
#tfidf highest keywords ffnn negative
get_keywords(ffnn_neutral.Tokens.values,'TFIDFFFNN_neutral_keywords.csv').head(20)

TFIDFFFNN_neutral_keywords.csv


,Token,TFIDF_Value
0,Ilaw,5.158883
519,pamunuan,5.158883
510,pagpunta,5.158883
511,pagtatag,5.158883
512,paid,5.158883
513,paint,5.158883
514,pakikiisa,5.158883
515,pakikinig,5.158883
516,palaring,5.158883
517,pamasahe,5.158883


In [15]:
#positive
ffnn_positive

,Tweet,Tokens,Label,Prediction
0,bbmsara win rin inday sara win,"[bbmsara, win, rin, inday, sara, win]",Positive,0.891507
3,gobyernong tapat angat buhay si yancy iskolar ...,"[gobyernong, tapat, angat, buhay, si, yancy, i...",Positive,0.998858
4,oras kandidatong tunay magdadala pagbabago nau...,"[oras, kandidatong, tunay, magdadala, pagbabag...",Positive,0.969080
7,gobyernong tapat angat buhay siyempre ipapanal...,"[gobyernong, tapat, angat, buhay, siyempre, ip...",Positive,0.999852
16,yung messenger puro pink dp yasss president cutie,"[yung, messenger, puro, pink, dp, yasss, presi...",Positive,0.959447
...,...,...,...,...
757,true leader shows mans best job woman vice pre...,"[true, leader, shows, mans, best, job, woman, ...",Positive,0.924789
762,ready all-out ba eto starter pack ipanalo natin,"[ready, all-out, ba, eto, starter, pack, ipana...",Positive,0.996817
763,presidente bise presidente kiko pangilinan let...,"[presidente, bise, presidente, kiko, pangilina...",Positive,0.998056
764,mt mayon showed seems like looking vp like vp ...,"[mt, mayon, showed, seems, like, looking, vp, ...",Positive,0.564144


In [16]:
#top ffnn hate sentences sorted by prediction
ffnn_positive_sortedbyprediction = ffnn_positive.sort_values(by=['Prediction'], ascending=False)
ffnn_positive_sortedbyprediction.to_csv('TFIDFFFNN_positive_results.csv', index=False)
ffnn_positive_sortedbyprediction

,Tweet,Tokens,Label,Prediction
61,student leader volunteer youth volunteer leni-...,"[student, leader, volunteer, youth, volunteer,...",Positive,1.000000
392,gobyernong tapat angat buhay si avril carat na...,"[gobyernong, tapat, angat, buhay, si, avril, c...",Positive,0.999999
621,si zai directioner boto kay girl almighty kasa...,"[si, zai, directioner, boto, kay, girl, almigh...",Positive,0.999999
515,si ady antuking estudyante nakiki-isa pagsupor...,"[si, ady, antuking, estudyante, nakiki-isa, pa...",Positive,0.999997
564,talaga natin tunay pagbabago kakayahan puso pa...,"[talaga, natin, tunay, pagbabago, kakayahan, p...",Positive,0.999993
...,...,...,...,...
251,president vice president kiko pangilinan napak...,"[president, vice, president, kiko, pangilinan,...",Positive,0.533971
434,presidente bise presidente kiko pangilinan,"[presidente, bise, presidente, kiko, pangilinan]",Positive,0.516793
188,heto si madam president congrats amazing,"[heto, si, madam, president, congrats, amazing]",Positive,0.508420
35,re-watching mmk story vp robredoandun part bum...,"[re-watching, mmk, story, vp, robredoandun, pa...",Positive,0.500418


In [17]:
#for negative
ffnn_negative

,Tweet,Tokens,Label,Prediction
2,presidential aspirant manila fired shots fello...,"[presidential, aspirant, manila, fired, shots,...",Negative,0.763399
5,ganyan nalang kontra sasabihin bbm kase wala y...,"[ganyan, nalang, kontra, sasabihin, bbm, kase,...",Negative,0.999668
6,dictator ferdinand leading polls become next p...,"[dictator, ferdinand, leading, polls, become, ...",Negative,0.999998
8,supporters warned vs sample ballots wrong assi...,"[supporters, warned, vs, sample, ballots, wron...",Negative,0.552879
9,smni debates matter nice debate biased shunga ...,"[smni, debates, matter, nice, debate, biased, ...",Negative,1.000000
...,...,...,...,...
740,wag nyo kausapin bibigyan nyo source galing com,"[wag, nyo, kausapin, bibigyan, nyo, source, ga...",Negative,0.628634
748,mas tiwala kay vp kesa kay narcos sksksks,"[mas, tiwala, kay, vp, kesa, kay, narcos, sksk...",Negative,0.625272
752,ngarag shunga inendorse niyo si bobong narcos ...,"[ngarag, shunga, inendorse, niyo, si, bobong, ...",Negative,0.999997
755,lutang magnanakaw boto magnanakaw pwesto gobye...,"[lutang, magnanakaw, boto, magnanakaw, pwesto,...",Negative,0.998900


In [18]:
#top ffnn negative sentences sorted by prediction
ffnn_negative_sortedbyprediction = ffnn_negative.sort_values(by=['Prediction'], ascending=False)
ffnn_negative_sortedbyprediction.to_csv('TFIDFFFNN_negative_results.csv', index=False)
ffnn_negative_sortedbyprediction

,Tweet,Tokens,Label,Prediction
221,toxic pinklawan impluwensya fushiang ina niyo,"[toxic, pinklawan, impluwensya, fushiang, ina,...",Negative,1.000000
664,bobo marunong ilocano attending debates arent ...,"[bobo, marunong, ilocano, attending, debates, ...",Negative,1.000000
157,marcosjr duwag ayaw umattend debate kayang ipa...,"[marcosjr, duwag, ayaw, umattend, debate, kaya...",Negative,1.000000
538,nagsasabing bobo si gago mas bobo kayo hahahah...,"[nagsasabing, bobo, si, gago, mas, bobo, kayo,...",Negative,1.000000
721,reading comments running president gosh apaka ...,"[reading, comments, running, president, gosh, ...",Negative,1.000000
...,...,...,...,...
8,supporters warned vs sample ballots wrong assi...,"[supporters, warned, vs, sample, ballots, wron...",Negative,0.552879
332,putang ina panalo kiko pangilinan leche bise p...,"[putang, ina, panalo, kiko, pangilinan, leche,...",Negative,0.545773
279,nko nanay sumama kna paloko kay bbm kita sinig...,"[nko, nanay, sumama, kna, paloko, kay, bbm, ki...",Negative,0.496415
95,voting loyalty continuing prrds legacy feeding...,"[voting, loyalty, continuing, prrds, legacy, f...",Negative,0.494369


In [19]:
#for neutral
ffnn_neutral

,Tweet,Tokens,Label,Prediction
17,see allows subjected public scrutiny present d...,"[see, allows, subjected, public, scrutiny, pre...",Neutral,0.983078
21,lang yung boboto kay compound dumami yung dati...,"[lang, yung, boboto, kay, compound, dumami, yu...",Neutral,0.997499
22,president kiko pangilinan vice president,"[president, kiko, pangilinan, vice, president]",Neutral,0.601136
26,best job woman president,"[best, job, woman, president]",Neutral,0.519654
28,right job woman,"[right, job, woman]",Neutral,0.486873
...,...,...,...,...
737,kumpara natin sagot,"[kumpara, natin, sagot]",Neutral,0.742125
741,presidential bets pick doh secretary via vp qu...,"[presidential, bets, pick, doh, secretary, via...",Neutral,0.979843
743,businessman advocated governmental shift feder...,"[businessman, advocated, governmental, shift, ...",Neutral,0.963517
759,president,[president],Neutral,0.534855


In [20]:
#top ffnn neutral sentences sorted by prediction
ffnn_neutral_sortedbyprediction = ffnn_neutral.sort_values(by=['Prediction'], ascending=False)
ffnn_neutral_sortedbyprediction.to_csv('TFIDFFFNN_neutral_results.csv', index=False)
ffnn_neutral_sortedbyprediction

,Tweet,Tokens,Label,Prediction
427,look caceres archbishop rolando tria tirona me...,"[look, caceres, archbishop, rolando, tria, tir...",Neutral,1.000000
450,gathering massive crowds different sorties vic...,"[gathering, massive, crowds, different, sortie...",Neutral,0.999998
58,reporter's review lian buan via,"[reporter's, review, lian, buan, via]",Neutral,0.999994
578,ibinida presidential contender senador panfilo...,"[ibinida, presidential, contender, senador, pa...",Neutral,0.999987
165,presidential candidate says willing release st...,"[presidential, candidate, says, willing, relea...",Neutral,0.999983
...,...,...,...,...
26,best job woman president,"[best, job, woman, president]",Neutral,0.519654
160,wait makapag freak properly nasa h2h pasay wai...,"[wait, makapag, freak, properly, nasa, h2h, pa...",Neutral,0.515449
320,one time presidente bise presidente kiko pangi...,"[one, time, presidente, bise, presidente, kiko...",Neutral,0.512662
210,lunes isigaw iboto presidente aksyon,"[lunes, isigaw, iboto, presidente, aksyon]",Neutral,0.511004


In [21]:
#For fasttext results

In [22]:
fastTextCNN_test_results.to_csv('fastTextCNN_test_results.csv', index=False)
fastTextCNN_test_results

,Tweet,Tokens,Label,Prediction
0,bbmsara win rin inday sara win,"[bbmsara, win, rin, inday, sara, win]",Positive,0.576204
1,inaniyo pag di si nanalo itatakwil pamilya,"[inaniyo, pag, di, si, nanalo, itatakwil, pami...",Negative,0.382223
2,presidential aspirant manila fired shots fello...,"[presidential, aspirant, manila, fired, shots,...",Negative,0.851224
3,gobyernong tapat angat buhay si yancy iskolar ...,"[gobyernong, tapat, angat, buhay, si, yancy, i...",Positive,0.979453
4,oras kandidatong tunay magdadala pagbabago nau...,"[oras, kandidatong, tunay, magdadala, pagbabag...",Positive,0.657396
...,...,...,...,...
762,ready all-out ba eto starter pack ipanalo natin,"[ready, all-out, ba, eto, starter, pack, ipana...",Positive,0.615797
763,presidente bise presidente kiko pangilinan let...,"[presidente, bise, presidente, kiko, pangilina...",Positive,0.780514
764,mt mayon showed seems like looking vp like vp ...,"[mt, mayon, showed, seems, like, looking, vp, ...",Positive,0.587240
765,president,[president],Neutral,0.698385


In [23]:
groupedby_cnn_label = fastTextCNN_test_results.groupby(fastTextCNN_test_results.Label)
#group by label
cnn_positive = groupedby_cnn_label.get_group("Positive")
cnn_negative = groupedby_cnn_label.get_group("Negative")
cnn_neutral = groupedby_cnn_label.get_group("Neutral")

In [24]:
#get top keywords by tfidf

In [25]:
#fasttextcnn highest keywords positive
get_keywords(cnn_positive.Tokens.values,'fastTextCNN_positive_keywords.csv').head(20)

fastTextCNN_positive_keywords.csv


,Token,TFIDF_Value
0,aaaah,5.255613
427,overflowing,5.255613
429,pack,5.255613
430,paet,5.255613
434,pagboboto,5.255613
435,pagiging,5.255613
436,pagkaka,5.255613
437,pagkakataon,5.255613
438,pagod,5.255613
439,pagpapaunlad,5.255613


In [26]:
#fasttextcnn highest keywords negative
get_keywords(cnn_negative.Tokens.values,'fastTextCNN_negative_keywords.csv').head(20)

fastTextCNN_negative_keywords.csv


,Token,TFIDF_Value
0,aattend,5.537961
962,nangyari,5.537961
970,napapa,5.537961
969,napakalaking,5.537961
968,napakagandang,5.537961
967,napakadaming,5.537961
966,napakadali,5.537961
965,nanood,5.537961
964,naniniwala,5.537961
963,naninira,5.537961


In [27]:
#fasttextcnn highest keywords neutral
get_keywords(cnn_neutral.Tokens.values,'fastTextCNN_neutral_keywords.csv').head(20)

fastTextCNN_neutral_keywords.csv


,Token,TFIDF_Value
0,Ilaw,5.477337
665,party,5.477337
636,pages,5.477337
637,pagkakaisa,5.477337
638,pagkampanya,5.477337
639,pagkapangulo,5.477337
640,pagma-migrant,5.477337
641,pagpunta,5.477337
642,pagsama,5.477337
643,pagtatag,5.477337


In [28]:
cnn_positive

,Tweet,Tokens,Label,Prediction
0,bbmsara win rin inday sara win,"[bbmsara, win, rin, inday, sara, win]",Positive,0.576204
3,gobyernong tapat angat buhay si yancy iskolar ...,"[gobyernong, tapat, angat, buhay, si, yancy, i...",Positive,0.979453
4,oras kandidatong tunay magdadala pagbabago nau...,"[oras, kandidatong, tunay, magdadala, pagbabag...",Positive,0.657396
7,gobyernong tapat angat buhay siyempre ipapanal...,"[gobyernong, tapat, angat, buhay, siyempre, ip...",Positive,0.925456
10,hidilyn diaz bringing home country's first oly...,"[hidilyn, diaz, bringing, home, country's, fir...",Positive,0.648636
...,...,...,...,...
757,true leader shows mans best job woman vice pre...,"[true, leader, shows, mans, best, job, woman, ...",Positive,0.812719
762,ready all-out ba eto starter pack ipanalo natin,"[ready, all-out, ba, eto, starter, pack, ipana...",Positive,0.615797
763,presidente bise presidente kiko pangilinan let...,"[presidente, bise, presidente, kiko, pangilina...",Positive,0.780514
764,mt mayon showed seems like looking vp like vp ...,"[mt, mayon, showed, seems, like, looking, vp, ...",Positive,0.587240


In [29]:
#top cnn positive sentences sorted by prediction
cnn_positive_sortedbyprediction = cnn_positive.sort_values(by=['Prediction'], ascending=False)
cnn_positive_sortedbyprediction.to_csv('fastTextCNN_positive_results.csv', index=False)
cnn_positive_sortedbyprediction

,Tweet,Tokens,Label,Prediction
464,gobyernong tapat angat buhay si rhoa matias bo...,"[gobyernong, tapat, angat, buhay, si, rhoa, ma...",Positive,0.981685
472,gobyernong tapat angat buhay si alvah student ...,"[gobyernong, tapat, angat, buhay, si, alvah, s...",Positive,0.981569
647,gobyernong tapat angat buhay si zeph engineeri...,"[gobyernong, tapat, angat, buhay, si, zeph, en...",Positive,0.981432
424,gobyernong tapat angat buhay si kaizen manunul...,"[gobyernong, tapat, angat, buhay, si, kaizen, ...",Positive,0.981288
84,"gobyernong tapat,angat buhay si jean army,naki...","[gobyernong, tapat,angat, buhay, si, jean, arm...",Positive,0.980205
...,...,...,...,...
767,lang mahikayat tao presidente si,"[lang, mahikayat, tao, presidente, si]",Positive,0.421576
16,yung messenger puro pink dp yasss president cutie,"[yung, messenger, puro, pink, dp, yasss, presi...",Positive,0.409448
352,unite filipinos like,"[unite, filipinos, like]",Positive,0.395981
359,first time voting sasayangin pagkakataon parte...,"[first, time, voting, sasayangin, pagkakataon,...",Positive,0.389150


In [30]:
cnn_negative

,Tweet,Tokens,Label,Prediction
1,inaniyo pag di si nanalo itatakwil pamilya,"[inaniyo, pag, di, si, nanalo, itatakwil, pami...",Negative,0.382223
2,presidential aspirant manila fired shots fello...,"[presidential, aspirant, manila, fired, shots,...",Negative,0.851224
5,ganyan nalang kontra sasabihin bbm kase wala y...,"[ganyan, nalang, kontra, sasabihin, bbm, kase,...",Negative,0.963385
9,smni debates matter nice debate biased shunga ...,"[smni, debates, matter, nice, debate, biased, ...",Negative,0.997610
12,pinapayagan nga magdala kodigo pag bumoboto ma...,"[pinapayagan, nga, magdala, kodigo, pag, bumob...",Negative,0.961129
...,...,...,...,...
750,susunod si former senator ferdinand deadma rin...,"[susunod, si, former, senator, ferdinand, dead...",Negative,0.526560
752,ngarag shunga inendorse niyo si bobong narcos ...,"[ngarag, shunga, inendorse, niyo, si, bobong, ...",Negative,0.991611
754,number red-tagging marites si emz hahaha,"[number, red-tagging, marites, si, emz, hahaha]",Negative,0.604467
755,lutang magnanakaw boto magnanakaw pwesto gobye...,"[lutang, magnanakaw, boto, magnanakaw, pwesto,...",Negative,0.973855


In [31]:
#top cnn negative sentences sorted by prediction
cnn_negative_sortedbyprediction = cnn_negative.sort_values(by=['Prediction'], ascending=False)
cnn_negative_sortedbyprediction.to_csv('fastTextCNN_negative_results.csv', index=False)
cnn_negative_sortedbyprediction

,Tweet,Tokens,Label,Prediction
538,nagsasabing bobo si gago mas bobo kayo hahahah...,"[nagsasabing, bobo, si, gago, mas, bobo, kayo,...",Negative,0.999998
721,reading comments running president gosh apaka ...,"[reading, comments, running, president, gosh, ...",Negative,0.999949
474,bobo talaga eh,"[bobo, talaga, eh]",Negative,0.999473
399,biggest flaws nating supporters eh yung pagigi...,"[biggest, flaws, nating, supporters, eh, yung,...",Negative,0.999455
722,nasabing takot si bbmarcos?!ng umattend si smn...,"[nasabing, takot, si, bbmarcos?!ng, umattend, ...",Negative,0.999264
...,...,...,...,...
222,po naniniwala deadlines tangina bagong motivat...,"[po, naniniwala, deadlines, tangina, bagong, m...",Negative,0.439242
593,asus wala nga si mr i-trend ninyo rin challeng...,"[asus, wala, nga, si, mr, i-trend, ninyo, rin,...",Negative,0.435528
542,pinakabaliw political supporters buong mundo d...,"[pinakabaliw, political, supporters, buong, mu...",Negative,0.427142
1,inaniyo pag di si nanalo itatakwil pamilya,"[inaniyo, pag, di, si, nanalo, itatakwil, pami...",Negative,0.382223


In [32]:
cnn_neutral

,Tweet,Tokens,Label,Prediction
11,ho ohhh madalas nyo sinasabi search muna bhe,"[ho, ohhh, madalas, nyo, sinasabi, search, mun...",Neutral,0.566759
17,see allows subjected public scrutiny present d...,"[see, allows, subjected, public, scrutiny, pre...",Neutral,0.505825
21,lang yung boboto kay compound dumami yung dati...,"[lang, yung, boboto, kay, compound, dumami, yu...",Neutral,0.436611
22,president kiko pangilinan vice president,"[president, kiko, pangilinan, vice, president]",Neutral,0.618976
24,dating sen nanguna presidential survey publicu...,"[dating, sen, nanguna, presidential, survey, p...",Neutral,0.768933
...,...,...,...,...
741,presidential bets pick doh secretary via vp qu...,"[presidential, bets, pick, doh, secretary, via...",Neutral,0.476914
753,want tarp sumn,"[want, tarp, sumn]",Neutral,0.428877
756,vice president final statement best job woman ...,"[vice, president, final, statement, best, job,...",Neutral,0.590370
759,president,[president],Neutral,0.698385


In [33]:
#top cnn neutral sentences sorted by prediction
cnn_neutral_sortedbyprediction = cnn_neutral.sort_values(by=['Prediction'], ascending=False)
cnn_neutral_sortedbyprediction.to_csv('fastTextCNN_neutral_results.csv', index=False)
cnn_neutral_sortedbyprediction

,Tweet,Tokens,Label,Prediction
556,sinimulan kaninang umaga tandem nina vice pres...,"[sinimulan, kaninang, umaga, tandem, nina, vic...",Neutral,0.958519
427,look caceres archbishop rolando tria tirona me...,"[look, caceres, archbishop, rolando, tria, tir...",Neutral,0.912471
129,look day religious group iglesia cristo endors...,"[look, day, religious, group, iglesia, cristo,...",Neutral,0.911720
374,announcement pdsp presidential candidate atten...,"[announcement, pdsp, presidential, candidate, ...",Neutral,0.910627
589,presidential candidate said filipino workers d...,"[presidential, candidate, said, filipino, work...",Neutral,0.902819
...,...,...,...,...
587,,[],Neutral,0.395895
30,,[],Neutral,0.395895
341,,[],Neutral,0.395895
486,bagsakan feat madam magalona hahahahahahaha,"[bagsakan, feat, madam, magalona, hahahahahahaha]",Neutral,0.383945
